In [1]:
import cv2
import os
import face_recognition
import numpy as np
import serial

In [2]:
ser = serial.Serial()
ser.port = 'COM3'
ser.baudrate = 9600
ser.bytesize = 8
ser.parity = serial.PARITY_NONE
ser.stopbits = serial.STOPBITS_ONE

In [3]:
cap = cv2.VideoCapture(0)

In [4]:
path = 'data'
stdImg = []
stdName = []
myList = os.listdir(path)

In [5]:
for cl in myList:
    curimg = cv2.imread(f'{path}/{cl}')
    stdImg.append(curimg)
    stdName.append(os.path.splitext(cl)[0])

In [6]:
studentName = [name.upper() for name in stdName]

In [7]:
print('Student Names in the List is :',studentName)

Student Names in the List is : ['DHYA', 'JAYA', 'KARTHICK', 'SANJAY', 'SATHISH']


In [8]:
stdname_copy = studentName.copy()

In [9]:
def resize(img, size) :
    width = int(img.shape[1]*size)
    height = int(img.shape[0] * size)
    dimension = (width, height)
    return cv2.resize(img, dimension, interpolation= cv2.INTER_AREA)

In [10]:
def findEncoding(images) :
    imgEncodings = []
    for img in images :
        img = resize(img, 0.50)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encodeimg = face_recognition.face_encodings(img)[0]
        imgEncodings.append(encodeimg)
    return imgEncodings

In [11]:
EncodeList = findEncoding(stdImg)

In [12]:
#in_camera = []

In [ ]:
while True :
    success, frame = cap.read()
    Smaller_frames = cv2.resize(frame, (0,0), None, 0.25, 0.25)

    facesInFrame = face_recognition.face_locations(Smaller_frames)
    encodeFacesInFrame = face_recognition.face_encodings(Smaller_frames, facesInFrame)
    in_camera = []
        
    for encodeFace, faceloc in zip(encodeFacesInFrame, facesInFrame) :
        matches = face_recognition.compare_faces(EncodeList, encodeFace)
        facedis = face_recognition.face_distance(EncodeList, encodeFace)
        matchIndex = np.argmin(facedis)
        #print(matchIndex)
        #print(matches[matchIndex])

        if matches[matchIndex] :
            name = studentName[matchIndex].upper()
            y1, x2, y2, x1 = faceloc
            y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
            cv2.rectangle(frame, (x1, y2-25), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            in_camera.append(name)
            if name == 'DHYA':
                b = b'E'
                ser.open()
                ser.write(b)
                ser.close()
            elif name == 'SATHISH':
                b = b'A'
                ser.open()
                ser.write(b)
                ser.close()
            elif name == 'KARTHICK':
                b = b'B'
                ser.open()
                ser.write(b)
                ser.close()
            elif name == 'JAYA':
                b = b'C'
                ser.open()
                ser.write(b)
                ser.close()
            elif name == 'SANJAY':
                b = b'D'
                ser.open()
                ser.write(b)
                ser.close()
        
    #print(in_camera)
    cv2.imshow('Face Attendance System',frame)
    k = cv2.waitKey(1)
    if k == 27:
        break
    
cap.release()
cv2.destroyAllWindows()